In [30]:
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd
from rasterstats import zonal_stats
import polars as pl

In [13]:
ds = xr.load_dataset("noaa_ds/nclimgrid_prcp.nc")

In [14]:
precip = ds["prcp"]

In [15]:
precip

<xarray.DataArray 'prcp' (time: 1568, lat: 596, lon: 1385)> Size: 5GB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]],
      shape=(1568, 596, 1385), dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 13kB 1895-01-01 1895-02-01 ... 2025-08-01
  * lat      (lat) float32 2kB 49.35 49.31 49.27 49.23 ... 24.65 24.6 24.56
  * lon      (lon) float32 6kB -124.7 -124.6 -124.6 ... -67.1 -67.06 -67.02
Attributes:
    references:     GHCN-Monthly Version 3 (Vose et al. 2011), NCEI/NOAA, htt...
    long_name:      Precipitation, monthly total
    standard_name:  precipitation_amount
    units:          millimeter
    valid_min:      0.0
    valid_max:      2000.0

In [17]:
precip = precip.rio.write_crs("EPSG:4326")

In [21]:
states = gpd.read_file("noaa_ds/us_states")

In [23]:
states = states.to_crs("EPSG:4326")

In [27]:
monthly_data = []

for t_index, t in enumerate(precip.time):
    # Select the precipitation data for this month
    prcp_month = precip.isel(time=t_index)
    
    # Convert to raster array for zonal_stats
    arr = prcp_month.values
    transform = prcp_month.rio.transform()
    
    # Compute zonal mean precipitation per state
    stats = zonal_stats(
        vectors=states.geometry,
        raster=arr,
        affine=transform,
        stats=['mean'],
        nodata=-9999
    )
    
    # Save results in DataFrame
    df = pd.DataFrame({
        'state': states['NAME'],  # adjust column if your shapefile uses a different name
        'date': pd.Timestamp(t.values),
        'precip_mm': [s['mean'] for s in stats]
    })
    
    monthly_data.append(df)

# Combine all months
monthly_precip_df = pd.concat(monthly_data, ignore_index=True)

In [28]:
monthly_precip_df.to_csv("noaa_ds/us_states_monthly_precip.csv", index=False)

In [31]:
monthly_precip_df = pl.from_pandas(monthly_precip_df)

In [32]:
monthly_precip_df

state,date,precip_mm
str,datetime[ns],f64
"""Maryland""",1895-01-01 00:00:00,105.736294
"""Iowa""",1895-01-01 00:00:00,21.822497
"""Delaware""",1895-01-01 00:00:00,114.367624
"""Ohio""",1895-01-01 00:00:00,95.695663
"""Pennsylvania""",1895-01-01 00:00:00,106.181771
…,…,…
"""Maine""",2025-08-01 00:00:00,49.252793
"""Hawaii""",2025-08-01 00:00:00,null
"""New Hampshire""",2025-08-01 00:00:00,35.12476


In [35]:
filtered = monthly_precip_df.filter(
    (pl.col("date") >= pl.datetime(2002, 1, 1)) &
    (pl.col("date") <= pl.datetime(2022, 12, 31))
)


In [36]:
filtered

state,date,precip_mm
str,datetime[ns],f64
"""Maryland""",2002-01-01 00:00:00,59.185694
"""Iowa""",2002-01-01 00:00:00,10.120294
"""Delaware""",2002-01-01 00:00:00,63.405518
"""Ohio""",2002-01-01 00:00:00,52.331947
"""Pennsylvania""",2002-01-01 00:00:00,53.991506
…,…,…
"""Maine""",2022-12-01 00:00:00,140.108649
"""Hawaii""",2022-12-01 00:00:00,null
"""New Hampshire""",2022-12-01 00:00:00,151.442007


In [38]:
filtered.write_csv("noaa_ds/us_states_monthly_precip_2002_2022.csv")